In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [4]:

# Step 1: Find all the CSV files in your current working directory
# The '*' acts as a wildcard, finding any file ending with '.csv'
all_csv_files = glob.glob("*.csv")

# Step 2: Read and combine all files into one DataFrame
# pd.concat stacks all the individual DataFrames found in the loop
df_master = pd.concat((
    pd.read_csv(f, encoding='unicode_escape', low_memory=False) 
    for f in all_csv_files
), ignore_index=True)

# Step 3: Display information about the new, combined DataFrame
print(f"Total files combined: {len(all_csv_files)}")
print("-" * 30)
print("First 5 rows of the master table:")
print(df_master.head())
print("-" * 30)
print(f"Total Rows in Master Data: {len(df_master):,}")

Total files combined: 2
------------------------------
First 5 rows of the master table:
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

      InvoiceDate  Price  Customer ID         Country  
0  12/1/2009 7:45   6.95      13085.0  United Kingdom  
1  12/1/2009 7:45   6.75      13085.0  United Kingdom  
2  12/1/2009 7:45   6.75      13085.0  United Kingdom  
3  12/1/2009 7:45   2.10      13085.0  United Kingdom  
4  12/1/2009 7:45   1.25      13085.0  United Kingdom  
------------------------------
Total Rows in Master Data: 1,067,371


In [9]:
#CHECKING DATA TYPE
df_master.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [10]:
# 1. Convert InvoiceDate to datetime object
df_master['InvoiceDate'] = pd.to_datetime(df_master['InvoiceDate'])

In [11]:
# 2. Convert Customer ID to string (it's an ID, not a number for math)
# We fill NaN with 0 for now just to make sure the type conversion works without error
# The next step will remove these rows anyway.
df_master['Customer ID'] = df_master['Customer ID'].astype(str)

In [13]:
# 3. Create the 'Revenue' column (Price * Quantity)
df_master['Revenue'] = df_master['Quantity']*df_master['Price']
df_master.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [15]:
# 4. Create a time-based feature (YearMonth) for trend analysis
df_master['YearMonth'] = df_master['InvoiceDate'].dt.to_period('M')
df_master.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,YearMonth
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009-12


In [16]:
# 5. Remove Canceled Orders
# Canceled transactions usually have a negative Quantity or the Invoice starts with 'C'
# We remove rows where Quantity is less than or equal to zero.

df_master = df_master[df_master['Quantity'] > 0]
print(f"Removed canceled orders. Remaining rows: {len(df_master):,}")

Removed canceled orders. Remaining rows: 1,044,421


In [17]:
# 6. Remove rows with Missing Customer IDs
# We check for 'nan' or empty string since we converted it to string earlier
df_master = df_master[df_master['Customer ID'].notna()]
df_master = df_master[df_master['Customer ID'] != 'nan']
print(f"Removed rows with missing Customer ID. Remaining rows: {len(df_master):,}")

Removed rows with missing Customer ID. Remaining rows: 805,620


In [18]:
# 7. Clean up the Description (Product Name)
# Remove leading/trailing spaces from product names
df_master['Description'] = df_master['Description'].str.strip()

In [19]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 805620 entries, 0 to 1067370
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805620 non-null  object        
 1   StockCode    805620 non-null  object        
 2   Description  805620 non-null  object        
 3   Quantity     805620 non-null  int64         
 4   InvoiceDate  805620 non-null  datetime64[ns]
 5   Price        805620 non-null  float64       
 6   Customer ID  805620 non-null  object        
 7   Country      805620 non-null  object        
 8   Revenue      805620 non-null  float64       
 9   YearMonth    805620 non-null  period[M]     
dtypes: datetime64[ns](1), float64(2), int64(1), object(5), period[M](1)
memory usage: 67.6+ MB


In [20]:
df_master.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Revenue        0
YearMonth      0
dtype: int64

In [21]:
# Top 10 Best-Selling Products (By Quantity)

top_selling_products = df_master.groupby('Description')['Quantity'].sum().nlargest(10)
print("\n--- Top 10 Best-Selling Products (By Quantity) ---")
print(top_selling_products)


--- Top 10 Best-Selling Products (By Quantity) ---
Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     109169
WHITE HANGING HEART T-LIGHT HOLDER     93640
PAPER CRAFT , LITTLE BIRDIE            80995
ASSORTED COLOUR BIRD ORNAMENT          79913
MEDIUM CERAMIC TOP STORAGE JAR         77916
JUMBO BAG RED RETROSPOT                75759
BROCADE RING PURSE                     71129
PACK OF 60 PINK PAISLEY CAKE CASES     55270
60 TEATIME FAIRY CAKE CASES            53495
PACK OF 72 RETROSPOT CAKE CASES        46107
Name: Quantity, dtype: int64


In [25]:
# Top 10 High-Revenue Generators

top_highest_revenue = df_master.groupby('Description')['Revenue'].sum().nlargest(10)
print("\n--- Top 10 High-Revenue Generators (By Revenue) ---")
print(top_highest_revenue.map('£{:,.2f}'.format)) # Formatting as currency for easy reading


--- Top 10 High-Revenue Generators (By Revenue) ---
Description
REGENCY CAKESTAND 3 TIER              £286,486.30
WHITE HANGING HEART T-LIGHT HOLDER    £252,072.46
PAPER CRAFT , LITTLE BIRDIE           £168,469.60
Manual                                £152,340.57
JUMBO BAG RED RETROSPOT               £136,980.08
ASSORTED COLOUR BIRD ORNAMENT         £127,074.17
POSTAGE                               £126,563.04
PARTY BUNTING                         £103,880.23
MEDIUM CERAMIC TOP STORAGE JAR         £81,416.73
PAPER CHAIN KIT 50'S CHRISTMAS         £79,594.33
Name: Revenue, dtype: object


In [30]:
#Sales Trend Over Time
sale_trens = df_master.groupby('YearMonth')['Revenue'].sum()
print("\n--- Monthly Revenue Trend ---")
print(sale_trens.map('£{:,.2f}'.format))


--- Monthly Revenue Trend ---
YearMonth
2009-12      £686,654.16
2010-01      £557,319.06
2010-02      £506,371.07
2010-03      £699,608.99
2010-04      £594,609.19
2010-05      £599,985.79
2010-06      £639,066.58
2010-07      £591,636.74
2010-08      £604,242.65
2010-09      £831,615.00
2010-10    £1,036,680.00
2010-11    £1,172,336.04
2010-12      £884,591.89
2011-01      £569,445.04
2011-02      £447,137.35
2011-03      £595,500.76
2011-04      £469,200.36
2011-05      £678,594.56
2011-06      £661,213.69
2011-07      £600,091.01
2011-08      £645,343.90
2011-09      £952,838.38
2011-10    £1,039,318.79
2011-11    £1,161,817.38
2011-12      £518,210.79
Freq: M, Name: Revenue, dtype: object


In [32]:
# Export the final clean DataFrame to a single CSV file
df_master.to_csv('cleaned_ecommerce_data_for_powerbi.csv', index=False)

print("\n--- Export Successful ---")
print("File 'cleaned_ecommerce_data_for_powerbi.csv' is ready to be loaded into Power BI.")


--- Export Successful ---
File 'cleaned_ecommerce_data_for_powerbi.csv' is ready to be loaded into Power BI.
